# Introduction

In the present short project, I investigate the current status of the electro charger infrastructure in Germany to get a short overview over the situation.<br>
Looking forward to buying maybe either an electric vehicle, an informed look on the situation wil help me to decide over the future acquisition.

Sources:
* Surface of each Land: <a href="https://www.statistikportal.de/de/bevoelkerung/flaeche-und-bevoelkerung">LINK </a>
* Registry of charging points (Ladesäulenregister): <a href="https://www.bundesnetzagentur.de/DE/Fachthemen/ElektrizitaetundGas/E-Mobilitaet/Ladesaeulenkarte/start.html">LINK</a>

<img src="../data/img/e-auto-ladestation-101.jpg" style="width:50%;height=50%"/>

# Load libaries & data

In [207]:
import pandas as pd
import plotly.graph_objects as go

In [208]:
df_charging_points = pd.read_csv("../data/Ladesaeulenregister.csv", sep=";", low_memory=False)
df_surface = pd.read_csv("../data/Gebietsfläche.csv",sep=";", low_memory=False)

# First look in the data

In [209]:
df_charging_points.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 69384 entries, 0 to 69383
Data columns (total 26 columns):
 #   Column                             Non-Null Count  Dtype 
---  ------                             --------------  ----- 
 0   Betreiber                          69381 non-null  object
 1   Straße                             69384 non-null  object
 2   Hausnummer                         69384 non-null  object
 3   Adresszusatz                       11780 non-null  object
 4   Postleitzahl                       69384 non-null  int64 
 5   Ort                                69384 non-null  object
 6   Bundesland                         69384 non-null  object
 7   Kreis/kreisfreie Stadt             69384 non-null  object
 8   Breitengrad                        69384 non-null  object
 9   Längengrad                         69384 non-null  object
 10  Inbetriebnahmedatum                69384 non-null  object
 11  Nennleistung Ladeeinrichtung [kW]  69384 non-null  object
 12  Art 

In [210]:
info_df = pd.DataFrame(index = df_charging_points.columns)
info_df = info_df.reset_index().rename({"index":"cols"}, axis = 1)

In [211]:
missing_values_abs = df_charging_points.isnull().sum().reset_index().rename({"index":"cols",0:"missing_abs"}, axis = 1)
missing_values_perc = df_charging_points.isnull().sum().reset_index().rename({"index":"cols", 0:"missing_perc"}, axis = 1)
nunique_vals = df_charging_points.nunique().reset_index().rename({"index":"cols", 0:"nunique_vals"}, axis = 1)
dtypes = df_charging_points.dtypes.reset_index().rename({"index":"cols",0:"dtype"}, axis = 1)

for df_to_join in [dtypes, nunique_vals, missing_values_abs, missing_values_perc]:

    info_df = pd.merge(left = info_df, right=df_to_join, left_on="cols", right_on="cols", how = "left")

In [212]:
info_df["mean_mode"] = [df_charging_points[col_].mode()[0] if dtype_=="object" else df_charging_points[col_].mean() for col_,dtype_ in zip(info_df["cols"], info_df["dtype"])]

In [213]:
info_df

,cols,dtype,nunique_vals,missing_abs,missing_perc,mean_mode
0,Betreiber,object,7748,3,3,EnBW mobility+ AG und Co.KG
1,Straße,object,24819,0,0,Hauptstraße
2,Hausnummer,object,2069,0,0,1
3,Adresszusatz,object,6045,57604,57604,Parkplatz
4,Postleitzahl,int64,6863,0,0,54644.057059
5,Ort,object,7081,0,0,Berlin
6,Bundesland,object,16,0,0,Bayern
7,Kreis/kreisfreie Stadt,object,401,0,0,Kreisfreie Stadt Berlin
8,Breitengrad,object,48793,0,0,"52,391331"
9,Längengrad,object,48959,0,0,"13,79215"


# Formulate general questions

I just want to have a rough understanding:
1) in which "Bundesland" (regional department) are provided how many charging stations?
2) Which city is the most advanced one concerning charging stations?
3) Which "Betreiber" (responsible company) are the top 5 provider?

# Answer the questions

## 1) In which "Bundesland" (regional department) are provided how many charging stations?


### Count of charging points per Bundesland

In [214]:
grouped_cnt = df_charging_points.groupby(["Bundesland", "Art der Ladeeinrichung"], as_index=False)["Betreiber"].count()
grouped_cnt.sort_values(by = ["Bundesland","Art der Ladeeinrichung"], ascending=True, inplace = True)

In [215]:
fig_land_cnt = go.Figure()

for art in grouped_cnt["Art der Ladeeinrichung"].unique():

    fig_land_cnt.add_trace(
        go.Bar(
            y = grouped_cnt[grouped_cnt["Art der Ladeeinrichung"] == art]["Bundesland"],
            x = grouped_cnt[grouped_cnt["Art der Ladeeinrichung"] == art]["Betreiber"],
            name = art,
            orientation="h"
        )
    )

fig_land_cnt.update_layout(
    barmode = "stack",
    autosize=False,
    width=800, 
    height=600,
    title="Count of Charging points per Bundesland (regional department)"
)

fig_land_cnt.show()

In [216]:
df_surface["Flaeche_qm"] = df_surface["Flaeche_qm"].apply(lambda x: x.replace(",", ".").replace(" ", ""))
df_surface["Flaeche_qm"] = df_surface["Flaeche_qm"].astype("float")

### Charging point per km²

In [217]:
grouped_cnt = pd.merge(left = grouped_cnt, right = df_surface, left_on = "Bundesland", right_on = "Bundesland", how = "left")
grouped_cnt["cnt_per_qm"] = grouped_cnt["Betreiber"]/grouped_cnt["Flaeche_qm"]

In [218]:
fig_land_cnt_per_qm = go.Figure()

for art in grouped_cnt["Art der Ladeeinrichung"].unique():

    fig_land_cnt_per_qm.add_trace(
        go.Bar(
            y = grouped_cnt[grouped_cnt["Art der Ladeeinrichung"] == art]["Bundesland"],
            x = grouped_cnt[grouped_cnt["Art der Ladeeinrichung"] == art]["cnt_per_qm"],
            name = art,
            orientation="h"
        )
    )

fig_land_cnt_per_qm.update_layout(
    barmode="stack",
    autosize=False,
    width=800, 
    height=600,
    title="Count of Charging points per km² per Bundesland (regional department)"
)

fig_land_cnt_per_qm.show()

## 2) Which city is the most advanced one concerning charging stations?

In [219]:
grouped_city_top_cnt = df_charging_points.groupby("Kreis/kreisfreie Stadt")["Betreiber"].count().sort_values(ascending=False).head()
grouped_city_top_cnt

Kreis/kreisfreie Stadt
Kreisfreie Stadt Berlin      2786
Kreisfreie Stadt München     1814
Stadtkreis Stuttgart         1556
Kreisfreie Stadt Hamburg     1397
Landkreis Region Hannover    1111
Name: Betreiber, dtype: int64

In [220]:
df_charging_points["Inbetriebnahmedatum"] = pd.to_datetime(df_charging_points["Inbetriebnahmedatum"], format="%d.%m.%Y")
df_charging_points["Inbetriebnahmedatum_month"] = df_charging_points["Inbetriebnahmedatum"] + pd.offsets.YearBegin(-1)

In [221]:
df_charging_points_timeline = df_charging_points.groupby(["Kreis/kreisfreie Stadt", "Inbetriebnahmedatum_month"], as_index = False)["Betreiber"].count().rename({"Betreiber": "cnt"}, axis = 1)
df_charging_points_timeline_top5 = df_charging_points_timeline[df_charging_points_timeline["Kreis/kreisfreie Stadt"].isin(grouped_city_top_cnt.index.tolist())]
df_charging_points_timeline_top5 = df_charging_points_timeline_top5[df_charging_points_timeline_top5["Inbetriebnahmedatum_month"].dt.year<2024]

In [222]:
fig_city_cnt = go.Figure()

for stadt in df_charging_points_timeline_top5["Kreis/kreisfreie Stadt"].unique():

    df_curr = df_charging_points_timeline_top5[df_charging_points_timeline_top5["Kreis/kreisfreie Stadt"] == stadt]

    fig_city_cnt.add_trace(
            go.Scatter(
                x = df_curr["Inbetriebnahmedatum_month"],
                y = df_curr["cnt"],
                name = stadt.split(" ")[-1],
                mode="lines+markers+text"
            )
        )

fig_city_cnt.update_layout(
    autosize=False,
    width=800, 
    height=600,
    title="Top 5 most advanced cities concerning charging stations"
)

fig_city_cnt.show()

## 3) Which "Betreiber" (responsible company) are the top 5 provider?

In [223]:
grouped_betreiber_top_cnt = df_charging_points.groupby("Betreiber")["Ort"].count().sort_values(ascending=False).head(10)
grouped_betreiber_top_cnt

Betreiber
EnBW mobility+ AG und Co.KG                                 3575
E.ON Drive GmbH                                             2283
Tesla Germany GmbH                                          1813
Mercedes-Benz AG                                            1308
EWE Go GmbH                                                 1141
ubitricity Gesellschaft für verteilte Energiesysteme mbH     899
Westenergie Metering GmbH                                    842
Audi AG                                                      834
HEnW-Mobil                                                   832
IONITY GmbH                                                  797
Name: Ort, dtype: int64

In [224]:
betreiber_timeline = df_charging_points.groupby(["Betreiber", "Inbetriebnahmedatum_month"], as_index = False)["Ort"].count().rename({"Ort": "cnt"}, axis = 1)
betreiber_timeline_top10 = betreiber_timeline[betreiber_timeline["Betreiber"].isin(grouped_betreiber_top_cnt.index.tolist())]
betreiber_timeline_top10 = betreiber_timeline_top10[betreiber_timeline_top10["Inbetriebnahmedatum_month"].dt.year<2024]

In [226]:
fig_betreiber_cnt = go.Figure()

for betreiber in betreiber_timeline_top10["Betreiber"].unique():

    df_curr = betreiber_timeline_top10[betreiber_timeline_top10["Betreiber"] == betreiber]

    fig_betreiber_cnt.add_trace(
            go.Scatter(
                x = df_curr["Inbetriebnahmedatum_month"],
                y = df_curr["cnt"],
                name = betreiber,
                mode="lines+markers+text"
            )
        )

fig_betreiber_cnt.update_layout(
    autosize=False,
    width=1200, 
    height=600,
    title="Top 10 most advanced Betreiber (resp. company) concerning charging stations"
)

fig_betreiber_cnt.show()

# Short Conclusion

The following insights could have been extracted:

1) in which "Bundesland" (regional department) are provided how many charging stations?
* Bayern, Baden-Württemberg and Nordrhein-Westfalen are the Top 3 departments setting up the most charging stations. Fast charger are significantly less implemented than "normal" charging stations. 
* The cities have (as expected) the highest density of charging points per km². 
2) Which city is the most advanced one concerning charging stations?
* Looking at the cities the most developed concerning this topic, Berlin, München and Stuttgart are the Top 3 cities.
* Looking at the timeline, Berlin and Stuttgart are in 2023 still on the rise. München on the other hand, slows down in the building of new charging points
3) Which "Betreiber" (responsible company) are the top 5 provider?
* EnBW, Eon, Tesla, Mercedes and EWE are the most active companies in setting up new charging points
* While Tesla, EnBW and EWE are poursuing their strong development, Mercedes and Eon are significantly slowing down their development.